1. Using AutoLoader to read the files data for orders
2. To union two streaming tables, we need to use Append Flows (Delta table and Auto Loader table)
3. We can use union, but it will read all the files and incremental logic will not work.
4. Adding a configuration, and using that to create gold tables


In [0]:
# DLT works with 3 types of datasets
# Streaming Tables (Permanent/Temporary) - used as Append Data Sources, Incremental data
# Materialized views - used for transformation, aggregations or computations
# Views - used for intermediate transformations, not stored in target schema

import dlt

In [0]:
_order_status = spark.conf.get("custom.orderStatus", "NA")

In [0]:
# create a streaming table for orders
@dlt.table(
  table_properties = {"quality" : "bronze"},
  comment = "Streaming table for orders bronze table"
)
def orders_bronze():
  df = spark.readStream.table("dev.bronze.orders_raw")
  return df

#@dlt.create_streaming_table(comment = "Streaming table for orders")
#def orders():
#  return spark.readStream.table("orders")*/

In [0]:
# create a streaming table for orders AutoLoader
@dlt.table(
  table_properties = {"quality" : "bronze"},
  comment = "Streaming table for orders AutoLoader",
  name = "orders_autoloader_bronze"
)
def func():
  df = (
      spark
      .readStream
      .format("cloudFiles")
      .option("cloudFiles.SchemaHints", "o_orderkey long, o_custkey long, o_orderstatus string, o_totalprice decimal(18,2), o_orderdate date, o_orderpriority string, o_clerk string, o_shippriority integer, o_comment string")
      .option("cloudFiles.schemaLocation", "/Volumes/dev/etl/landing/autoloader/schemas/1/")
      .option("cloudFiles.format", "csv")
      .option("pathGlobFilter", "*.csv")
      .option("cloudFilesSchemaEvolutionMode", "none")
      .load("/Volumes/dev/etl/landing/files/") 
    )  
  return df

#@dlt.create_streaming_table(comment = "Streaming table for orders")
#def orders():
#  return spark.readStream.table("orders")*/

In [0]:
dlt.create_streaming_table("orders_union_bronze")

#Append flow - First Table
@dlt.append_flow(
    target = "orders_union_bronze"
)
def order_delta_append():
    df = spark.readStream.table("LIVE.orders_bronze")
    return df

#Append flow  - Second Table
@dlt.append_flow(
    target = "orders_union_bronze"
)
def order_autoloader_append():
    df = spark.readStream.table("LIVE.orders_autoloader_bronze")
    return df


In [0]:
# create a materialized views for customers
# create a streaming table for orders
@dlt.table(
  table_properties = {"quality" : "bronze"},
  comment = "Materliazed view for customer bronze table",
  name = "customer_bronze"
)
def cust_bronze():
  df = spark.read.table("dev.bronze.customer_raw")
  return df


#@dlt.create_view(comment = "Materialized view for customers")
#def customers():
#  return spark.readStream.table("customers")
# create a view for products
#@dlt.create_view(comment = "View for products")
#def products():
#  return spark.readStream.table("products")
# create a view for sales
#@dlt.create_view(comment = "View for sales")
#def sales():
#  return spark.readStream.table("sales")

In [0]:
# create a viewto join orders with customers
@dlt.view(
    comment = "Bronze Joined table"
)
def joined_vw():
  df_c = spark.read.table("LIVE.customer_bronze")
  df_o = spark.read.table("LIVE.orders_union_bronze")

  df_join = df_o.join(df_c, how = "left_outer", on = df_o.o_custkey == df_c.c_custkey)
  return df_join


In [0]:
# create MV to add new column
from pyspark.sql.functions import current_timestamp

@dlt.table(
  table_properties = {"quality" : "silver"},
  comment = "joined table",
  name = "orders_silver"
)
def orders_silver():
  df = spark.read.table("LIVE.joined_vw").withColumn("__insert_date", current_timestamp())
  return df

In [0]:
# Aggregate based on c_mktsegment and find the count of order (c_orderkey)
from pyspark.sql.functions import current_timestamp, count, sum

@dlt.table(
  table_properties = {"quality" : "gold"},
  comment = "orders aggregated able"
)
def orders_agg_gold():
  df = spark.read.table("LIVE.orders_silver")

  df_final = df.groupBy("c_mktsegment").agg(count("o_orderkey").alias("count_orders"),sum("o_totalprice").alias("sum_totalprice")).withColumn("__insert_date", current_timestamp())

  return df_final

In [0]:
https://adb-7405616295561001.1.azuredatabricks.net/editor/notebooks/4055779692625112?contextId=pipeline%3A34865939-5be6-4478-bd91-0c0288a607b8&o=7405616295561001$0for _status in _order_status.split(","):
    @dlt.table(
    table_properties = {"quality" : "gold"},
    comment = "orders aggregated able",
    name = f"orders_agg_{_status}_gold"
    )
    def orders_agg_gold():
       df = spark.read.table("LIVE.orders_silver")

       df_final = df.where(f"o_orderstatus = '{_status}'").groupBy("c_mktsegment").agg(count("o_orderkey").alias("count_orders"),sum("o_totalprice").alias("sum_totalprice")).withColumn("__insert_date", current_timestamp())

       return df_final
